In [1]:
# Librerías necesarias
import requests
from bs4 import BeautifulSoup
import time
import csv
import json
import random
import re
from pymongo import MongoClient

# Tiempo al inicio de ejecutar el código: para saber el tiempo total de ejecucción
inicio = time.time()

# URL de idealista sin filtrar por ciudad
base_url = 'https://www.idealista.com'

# Encabezados para simular un navegador real y evitar bloqueos
headers_list = [
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
        'Accept-Language': 'es-ES,es;q=0.9',
        'Referer': 'https://www.google.com/'
    },
    {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:115.0) Gecko/20100101 Firefox/115.0',
        'Accept-Language': 'es-ES,es;q=0.8',
        'Referer': 'https://www.bing.com/'
    },
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.94 Safari/537.36 Edg/122.0.0.0',
        'Accept-Language': 'es-ES,es;q=0.7',
        'Referer': 'https://duckduckgo.com/'
    },
    {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 10; SM-G970F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36',
        'Accept-Language': 'es-ES,es;q=0.9',
        'Referer': 'https://www.google.com/'
    },
    {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15',
        'Accept-Language': 'es-ES,es;q=0.9',
        'Referer': 'https://www.ecosia.org/'
    }
]


# URLs que se van a scrapear, en cada una de filtra por venta, Córdoba y un tipo de situación de vivienda. Se hace así porque ese filtro no se puede sacar del propio anuncio
urls_y_situaciones = {
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-nuda-propiedad,inquilino,ocupado/': 'Nuda propiedad, Alquilada, Ocupada',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-estudios,de-un-dormitorio,sin-inquilinos/': 'Disponible',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-de-dos-dormitorios,sin-inquilinos/': 'Disponible',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-de-tres-dormitorios,sin-inquilinos/': 'Disponible',
    'https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-de-cuatro-cinco-habitaciones-o-mas,sin-inquilinos/': 'Disponible'
}

# Inicializamos la lista donde se guardarán los anuncios
resultados = []

# Los anuncios obtenidos se guardarán en fichero tipo csv y json, para posteriormente elegir el mejor formato

# Abrimos archivo CSV para guardar los resultados estructurados, mode='w' porque es modo escritura y lo reescribe de 0, newline='' para evitar escribir lineas en blanco entre lineas de csv y codificación utf-8
with open('idealista_resultados1.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Nombres de cabecera del csv
    writer.writerow([
        'Título', 'Precio', 'Tipo de vivienda', 'Zona', 'Habitaciones', 'Metros cuadrados',
        'Baños', 'Planta', 'Estado', 'Situación vivienda', 'Tipo de anuncio', 'Multimedia',
        'Aire acondicionado', 'Armarios empotrados', 'Ascensor', 'Balcón o terraza',
        'Exterior', 'Garaje', 'Jardín', 'Piscina', 'Trastero', 'Vivienda accesible',
        'Vivienda de lujo', 'Descripción',
        'Metros cabecera',
        'Orientación', 'Construido en', 'Acceso movilidad', 'Calefacción',
        'Consumo energético', 'Emisiones', 'Equipamiento extra'
    ])
    
    # Recorremos cada una de las URLs con su tipo de situación
    for base_list_url, situacion_fijada in urls_y_situaciones.items():


        # Construimos la URL combinando situación y habitaciones
        url_modificada = base_list_url
        # Se imprime un texto en negrita del tipo de situación que se va a scrapear
        print(f"\033[1m\nSe comienza a scrapear: {situacion_fijada}\033[0m")
        
        # Inicializamos que estamos en la página 1 de ese tipo de situación, porque luego habrá que recorrer todas las páginas
        pagina = 1
        
        while True:
            if pagina == 1: # Si estamos en la página 1 cogemos la URL base que siempre lleva a la página 1
                list_url = url_modificada 
            else:#Si no estamos en la 1 hay que modificar la URL para que busque en la página correspondiente
                list_url = url_modificada.rstrip('/') + f'/pagina-{pagina}.htm' 
                
            print(f"\nScrapeando el tipo de situación: {situacion_fijada} en la URL: {list_url}")

            #Función de la librería requests para hacer web scraping, eligiendo el header aleatoriamente
            headers = random.choice(headers_list)
            response = requests.get(list_url, headers=headers)
            
            #Si se obtiene un error al acceder a la página se sale del bucle        
            if response.status_code != 200:
                print("Se ha obtenido un error al acceder a la URL, error: ", response.status_code)
                break

            #Analizar código HTML para extraer información de él
            soup = BeautifulSoup(response.content, 'html.parser')
            listings = soup.find_all('article')

            #Si no hay anuncios se sale del bucle
            if not listings:
                print("sin anuncios")
                break

            #Se guarda la URL del primer anuncio que sale en la URL buscada    
            first_listing = listings[0].find('a', class_='item-link')
            url_anuncio_actual = first_listing['href'] if first_listing and first_listing.has_attr('href') else None

            # Guardamos la URL del primer anuncio de la primera página, porque si una página no existe (por ejemplo, la página 99), el sitio web redirige automáticamente a la página 1.
            # Comparando la URL del primer anuncio de cada página con la de la página 1, podemos saber si hemos llegado al final del scrapeo.
            if pagina == 1:
                url_anuncio_primera_pagina = url_anuncio_actual#solo se compara con el primer anuncio de la primera pagian, proque cuando la pagina no existe redirige a la primera
            else:
                if url_anuncio_actual == url_anuncio_primera_pagina:
                    print(f"La página no existe y se ha redirigido a la 1, scrapeo de la situación: {situacion_fijada} ha finalizado")
                    break

            # Imprimimos el número de anuncios que hay en cada página
            print(f" Hay {len(listings)} anuncios en la página {pagina}")

            # Iteramos por todos los anuncios obtenidos
            for listing in listings:

                # Inicialización de los campos a extraer de cada anuncio
                title = price = tipo_vivienda = situacion = habitaciones_anuncio = metros_anuncio = baños = planta_anuncio = estado = situacion_vivienda = tipo_anuncio = descripcion  = 'N/A'
                metros_construidos = orientacion = construido_en = acceso_movilidad = calefaccion = consumo_energetico = emisiones_energeticas = 'N/A'
                caracteristicas = equipamiento_extra = []
                multimedia = "Sin fotos"
                situacion_vivienda = situacion_fijada
                aire_acondicionado = armarios = ascensor = balcon = exterior = garaje = jardin = piscina = trastero = accesible = lujo = 'No'

                # Para cada anuncio se extrae: el título, el precio y el link
                title_tag = listing.find('a', class_='item-link')
                title = title_tag.get_text(strip=True) if title_tag else 'N/A'
                if title == 'N/A':
                    continue
                price_tag = listing.find('span', class_='item-price')
                if price_tag:
                    price_text = price_tag.get_text(strip=True)
                    match = re.search(r'[\d.]+', price_text)
                    if match:
                        # Elimina puntos y convierte a número
                        price = match.group().replace('.', '')
                    else:
                        price = 'N/A'
                else:
                    price = 'N/A'
                link = base_url + title_tag['href'] if title_tag and title_tag.has_attr('href') else None
                
                # Si el enlace es válido, lanzamos una nueva consulta al HTML del anuncio
                if link:
                    anuncio_res = requests.get(link, headers=headers)
                    if anuncio_res.status_code == 200:
                        anuncio_soup = BeautifulSoup(anuncio_res.content, 'html.parser')

                        title_tag = anuncio_soup.find('title')

                        
                        #Según los patrones que tenga el título obtenemos el tipo de vivienda
                        if title_tag:
                            title_text = title_tag.get_text(strip=True).lower()
                            if "piso" in title_text:
                                tipo_vivienda = "Piso"
                            elif "ático" in title_text:
                                tipo_vivienda = "Ático"
                            elif "dúplex" in title_text or "duplex" in title_text:
                                tipo_vivienda = "Dúplex"
                            elif "independiente" in title_text:
                                tipo_vivienda = "Casa o chalet independiente"
                            elif "pareado" in title_text:
                                tipo_vivienda = "Chalet pareado"
                            elif "adosado" in title_text:
                                tipo_vivienda = "Chalet adosado"
                            elif "rústica" in title_text or "cortijo" in title_text or "rural" in title_text:
                                tipo_vivienda = "Casa rústica"
                            elif "estudio" in title_text:
                                tipo_vivienda = "Estudio"

                        # Se obtiene la localización
                        location_tag = anuncio_soup.find('span', class_='main-info__title-minor')
                        situacion = location_tag.get_text(strip=True) if location_tag else 'N/A'

                        # De la cabecera del anuncio sacamos los metros cuadrados, el número de habitaciones y la planta
                        cabecera = anuncio_soup.find_all('div', class_='info-features')
                        if cabecera:
                            cab_texts = cabecera[0].get_text(" | ", strip=True).split(" | ")
                            for text in cab_texts:
                                if "m²" in text:
                                    match = re.search(r'\d+', text)
                                    metros_anuncio = match.group() if match else 'N/A'
                                elif "hab" in text:
                                    match = re.search(r'\d+', text)
                                    habitaciones_anuncio = match.group() if match else 'N/A'
                                elif "Planta" in text:
                                    planta_anuncio = text

                        # Se obtiene la descripción
                        desc_tag = anuncio_soup.find('div', {'class': 'adCommentsLanguage'})
                        descripcion = desc_tag.get_text(strip=True) if desc_tag else 'Sin descripción'

                        #Se obtienen el resto de caracteristicas del anuncio
                        caracteristicas_raw = anuncio_soup.find_all('li')
                        for li in caracteristicas_raw:
                            texto = li.get_text(strip=True)
                            caracteristicas.append(texto)

                            if "m² construidos" in texto:
                                metros_construidos = texto
                            elif "baño" in texto:
                                match = re.search(r'\d+', texto)
                                baños = match.group() if match else 'N/A'
                            elif "Planta" in texto:
                                planta = texto
                            elif "segunda mano" in texto.lower() or "nuevo" in texto.lower():
                                estado = texto
                            elif "orientación" in texto.lower():
                                orientacion = texto
                            elif "construido en" in texto.lower():
                                construido_en = texto
                            elif "movilidad reducida" in texto.lower():
                                acceso_movilidad = texto
                            elif "calefacción" in texto.lower():
                                calefaccion = texto
                            elif "consumo:" in texto.lower():
                                consumo_energetico = texto
                            elif "emisiones:" in texto.lower():
                                emisiones_energeticas = texto
                            elif "aire acondicionado" in texto.lower():
                                aire_acondicionado = 'Sí'
                            elif "armarios empotrados" in texto.lower():
                                armarios = 'Sí'
                            elif "ascensor" in texto.lower():
                                ascensor = 'Sí'
                            elif "balcón" in texto or "terraza" in texto.lower():
                                balcon = 'Sí'
                            elif "exterior" in texto.lower():
                                exterior = 'Sí'
                            elif "garaje" in texto.lower():
                                garaje = 'Sí'
                            elif "jardín" in texto.lower():
                                jardin = 'Sí'
                            elif "piscina" in texto.lower():
                                piscina = 'Sí'
                            elif "trastero" in texto.lower():
                                trastero = 'Sí'
                            elif "accesible" in texto.lower():
                                accesible = 'Sí'
                            elif "lujo" in texto.lower():
                                lujo = 'Sí'

                        # Recorre todos los <div> del anuncio buscando un encabezado (<h2> o <h3>) que contenga la palabra “equipamiento”, dentro busca una lista <ul> y dentro los <li>
                        for div in anuncio_soup.find_all('div'):
                            heading = div.find(['h2', 'h3'])
                            if heading and 'equipamiento' in heading.get_text(strip=True).lower():
                                ul = div.find('ul')
                                if ul:
                                    for li in ul.find_all('li'):
                                        texto = li.get_text(strip=True)
                                        if texto and texto not in equipamiento_extra:
                                            equipamiento_extra.append(texto)
                                break

                        # Bloque para anlizar si el anuncio tiene imágenes
                        imagenes = anuncio_soup.find_all('img', src=True)
                        fotos_reales = [
                            img['src'] for img in imagenes
                            if any(palabra in img['src'] for palabra in ['image.master', 'img4.idealista.com', 'foto'])
                        ]
                        multimedia = "Con fotos" if fotos_reales else "Sin fotos"

                        # Para saber si es de un particular o profesional
                        tipo_tag = anuncio_soup.find('div', class_='professional-name')
                        tipo_anuncio = 'Profesional' if tipo_tag else 'Particular'

                    #Espera de 1 segundo entre cada petición al servidor de Idealista. Para no parecer un robot y evitar ser bloqueado.
                    time.sleep(random.uniform(20,40))

                # Guarda los datos de cada anuncio en el csv
                writer.writerow([
                    title, price, tipo_vivienda, situacion, habitaciones_anuncio, metros_anuncio,
                    baños, planta_anuncio, estado, situacion_vivienda, tipo_anuncio,
                    multimedia, aire_acondicionado, armarios, ascensor, balcon, exterior,
                    garaje, jardin, piscina, trastero, accesible, lujo, descripcion,
                    metros_construidos, orientacion, construido_en, acceso_movilidad,
                    calefaccion, consumo_energetico, emisiones_energeticas, ", ".join(equipamiento_extra)
                ])

                # Guardar los datos de cada anuncio en el json
                resultados.append({
                    "Título": title, "Precio": price, "Tipo de vivienda": tipo_vivienda, "Zona": situacion,
                    "Habitaciones": habitaciones_anuncio, "Metros cuadrados": metros_anuncio, "Baños": baños, "Planta": planta_anuncio,
                    "Estado": estado, "Situación vivienda": situacion_vivienda, "Tipo de anuncio": tipo_anuncio, "Multimedia": multimedia,
                    "Aire acondicionado": aire_acondicionado, "Armarios empotrados": armarios,
                    "Ascensor": ascensor, "Balcón o terraza": balcon, "Exterior": exterior,
                    "Garaje": garaje, "Jardín": jardin, "Piscina": piscina, "Trastero": trastero,
                    "Vivienda accesible": accesible, "Vivienda de lujo": lujo, "Descripción": descripcion,
                    "Metros construidos/utiles": metros_construidos,
                    "Orientación": orientacion, "Construido en": construido_en,
                    "Acceso movilidad": acceso_movilidad, "Calefacción": calefaccion,
                    "Consumo energético": consumo_energetico, "Emisiones": emisiones_energeticas,
                    "Equipamiento extra": equipamiento_extra
                })

            # Se suma 1 al número de páginas para visitarlas todas
            time.sleep(random.uniform(200, 300))
            pagina += 1

# Crea el json para guardar todos los resultados
with open('idealista_resultados1.json', 'w', encoding='utf-8') as json_file:
    json.dump(resultados, json_file, ensure_ascii=False, indent=4)

#Insertar datos en MongoDB

# Conexión local, cambia el URI si tu MongoDB está en Atlas u otro lugar
client = MongoClient('mongodb://localhost:27017/')
db = client['Idealista']
collection = db['Anuncios_Cordoba']

#Borrar la colección existente
collection.delete_many({})

# Inserta todos los resultados en la colección
collection.insert_many(resultados)

# Mensaje avisando que se han guardado todos los resultados
print("\n Datos guardados en idealista_resultados1.csv y idealista_resultados1.json")

#Tiempo al finalizar de ejecutar el código: para saber el tiempo total de ejecucción
fin = time.time()
duracion = fin - inicio

# Mensaje para saber el tiempo total que tarda en scrapear la página
print(f"\n Tiempo total de ejecución: {duracion:.2f} segundos")


Se comienza a scrapear: Nuda propiedad

Scrapeando el tipo de situación: Nuda propiedad en la URL: https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-nuda-propiedad/
 Hay 18 anuncios en la página 1

Scrapeando el tipo de situación: Nuda propiedad en la URL: https://www.idealista.com/venta-viviendas/cordoba-cordoba/con-nuda-propiedad/pagina-2.htm
La página no existe y se ha redirigido a la 1, scrapeo de la situación: Nuda propiedad ha finalizado

 Datos guardados en idealista_resultados1.csv y idealista_resultados1.json

 Tiempo total de ejecución: 739.89 segundos
